# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [7]:
from azureml.core import Workspace, Experiment,Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

import azureml.core
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

import logging
import os
import joblib
import pandas as pd
import numpy as np

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace and Create an Azure ML experiment

In [2]:
ws = Workspace.from_config()
experiment_name = 'heart-failure-automl-experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-135902
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-135902


## Dataset

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
dataset_key = "Heart-Failure-Prediction-Dataset"
description_text = "Heart Failure Prediction Dataset"

if dataset_key in ws.datasets.keys(): 
    dataset = ws.datasets[dataset_key]
    print("Found dataset")
else:
    # Create AML Dataset and register it into Workspace
    dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(dataset_url)        
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                                name=dataset_key,
                                description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Found dataset


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Create or Attach an AmlCompute cluster

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpu-cluster"

# Check if the compute target exists
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=5)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2021-01-24T14:59:14.660177+00:00', 'createdBy': {'userObjectId': 'de62da4e-e66d-430d-a4a9-1c7ffdd05d75', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 135902'}, 'modifiedTime': '2021-01-24T15:02:16.464259+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS12_V2'}


## Training Configuration

In [5]:
train, test = train_test_split(df, random_state=0, test_size=0.2)
print(train.head(3))
print(train.shape)
train.to_csv('train.csv', index=False)

      age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
134  81.0        0                      4540         0                 35   
145  50.0        0                       185         0                 30   
63   45.0        0                       582         0                 35   

     high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
134                    0   231000.0              1.18           137    1   
145                    0   266000.0              0.70           141    1   
63                     0   385000.0              1.00           145    1   

     smoking  time  DEATH_EVENT  
134        1   107            0  
145        1   112            0  
63         0    61            1  
(239, 13)


In [6]:
datastore = ws.get_default_datastore()
print(datastore)
datastore.upload_files(files = ['./train.csv'])
train = Dataset.Tabular.from_delimited_files(path = [(datastore,'train.csv')])

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-9ef76144-e983-40a6-8655-4e4eab8b4c95",
  "account_name": "mlstrg135902",
  "protocol": "https",
  "endpoint": "core.windows.net"
}
Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
automl_settings = {
    "experiment_timeout_minutes" : 30,
    "iteration_timeout_minutes": 5,
    "max_cores_per_iteration": -1,
    "max_concurrent_iterations": 4,
    "enable_early_stopping" : True,
    "primary_metric": 'accuracy',
    "n_cross_validations": 3,
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    task="classification",
    compute_target=compute_target,
    training_data= train,
    label_column_name="DEATH_EVENT",
    debug_log = 'automl_errors.log',
    **automl_settings)

In [9]:
#Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-automl-experiment,AutoML_1d6b790b-08ce-4bcf-b70c-aa121e567c60,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
remote_run.wait_for_completion(show_output=True)


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_1d6b790b-08ce-4bcf-b70c-aa121e567c60',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-24T16:16:33.962838Z',
 'endTimeUtc': '2021-01-24T16:41:36.340577Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-automl-experiment","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-135902","workspace_name":"quick-starts-ws-135902","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"nu

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_automl_best_model = remote_run.get_output()
best_automlrun_metrics=best_run.get_metrics()
print('Best Run Id:', best_run.id, sep='\n')
print('Best Run Accuracy:', best_automlrun_metrics['accuracy'], sep='\n')
print('Best Run Metrics:', best_run.get_metrics(), sep='\n')
print('Best Run Properties:', best_run.get_properties(), sep='\n')
print('Best Model:', best_run.properties['model_name'], sep='\n')
print('Fitted Automl Best Model:', fitted_automl_best_model, sep='\n')

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Best Run Id:
AutoML_1d6b790b-08ce-4bcf-b70c-aa121e567c60_55
Best Run Accuracy:
0.8828059071729957
Best Run Metrics:
{'precision_score_micro': 0.8828059071729957, 'weighted_accuracy': 0.9081364376694537, 'recall_score_macro': 0.8486446039835869, 'precision_score_macro': 0.8736333579197093, 'precision_score_weighted': 0.8860926049300466, 'AUC_weighted': 0.9154698689048972, 'f1_score_macro': 0.8562924513053473, 'average_precision_score_micro': 0.9267888449231131, 'f1_score_weighted': 0.8806437875095002, 'recall_score_micro': 0.8828059071729957, 'f1_score_micro': 0.8828059071729957, 'balanced_accuracy': 0.8486446039835869, 'matthews_correlation': 0.720977865970507, 'recall_score_weighted': 0.8828059071729957, 'log_loss': 0.3543497980830313, 'average_precision_score_macro': 0.8947097922892953, 'average_precision_score_weighted': 0.9220067110057949, 'AUC_micro': 0.9257656859210597, 'norm_macro_recall': 0.6972892079671741, 'AUC_macro': 0.9154698689048972, 'accuracy': 0.8828059071729957, 'accu

In [14]:
for primary_metric in best_automlrun_metrics:
    metric=best_automlrun_metrics[primary_metric]
    print(primary_metric,metric)

precision_score_micro 0.8828059071729957
weighted_accuracy 0.9081364376694537
recall_score_macro 0.8486446039835869
precision_score_macro 0.8736333579197093
precision_score_weighted 0.8860926049300466
AUC_weighted 0.9154698689048972
f1_score_macro 0.8562924513053473
average_precision_score_micro 0.9267888449231131
f1_score_weighted 0.8806437875095002
recall_score_micro 0.8828059071729957
f1_score_micro 0.8828059071729957
balanced_accuracy 0.8486446039835869
matthews_correlation 0.720977865970507
recall_score_weighted 0.8828059071729957
log_loss 0.3543497980830313
average_precision_score_macro 0.8947097922892953
average_precision_score_weighted 0.9220067110057949
AUC_micro 0.9257656859210597
norm_macro_recall 0.6972892079671741
AUC_macro 0.9154698689048972
accuracy 0.8828059071729957
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_1d6b790b-08ce-4bcf-b70c-aa121e567c60_55/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_1d6b790b-08ce-4bcf-b70c-aa121e567

In [15]:
#Save the best model
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_automl_best_model, filename='outputs/automl.joblib')
print(best_run)

Run(Experiment: heart-failure-automl-experiment,
Id: AutoML_1d6b790b-08ce-4bcf-b70c-aa121e567c60_55,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [17]:
description = 'Heart Failure Prediction AutoML Model'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)
print(model.name, model.id, model.version, sep='\t')

AutoML1d6b790b055	AutoML1d6b790b055:1	1


In [18]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='inference/score.py')

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'type': "automl_classification"}, 
                                               description = 'Heart Failure Prediction AutoML Service')

deploy_service_name = 'automl-model-deployment'
service = Model.deploy(ws, deploy_service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

# Enable Application Insights
service.update(enable_app_insights=True)

print('Service State:', service.state, sep='\n')
print('Service Scoring URI:', service.scoring_uri, sep='\n')
print('Service Swagger URI:', service.swagger_uri, sep='\n')

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service State:
Healthy
Service Scoring URI:
http://357c9640-97d9-4585-a0be-dd03c068da9f.southcentralus.azurecontainer.io/score
Service Swagger URI:
http://357c9640-97d9-4585-a0be-dd03c068da9f.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:
import requests
import json

scoring_uri = service.scoring_uri

data = {"data":
        [
          { 
            "age": 80,
            "anaemia": 1,
            "creatinine_phosphokinase": 146,
            "diabetes": 1,
            "ejection_fraction": 20,
            "high_blood_pressure": 0,
            "platelets": 127000,
            "serum_creatinine": 9.4,
            "serum_sodium": 137,
            "sex": 1,
            "smoking": 1,
            "time": 104
          },
          {
            "age": 42,
            "anaemia": 1,
            "creatinine_phosphokinase": 111,
            "diabetes": 0,
            "ejection_fraction": 38,
            "high_blood_pressure": 1,
            "platelets": 87000,
            "serum_creatinine": 0.8,
            "serum_sodium": 116,
            "sex":0,
            "smoking": 0,
            "time": 13
          },
      ]
    }

input_data = json.dumps(data)
headers = {'Content-Type': 'application/json'}
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [20]:
print("\nService Logs:\n",service.get_logs())


Service Logs:
 2021-01-24T16:49:32,402406143+00:00 - iot-server/run 
2021-01-24T16:49:32,402762442+00:00 - gunicorn/run 
2021-01-24T16:49:32,407710924+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-24T16:49:32,402762442+00:00 - rsysl

## Cleanup

In [ ]:
service.delete()
model.delete()
compute_target.delete()